See the article __[Extracting Data from Wikidata Using SPARQL and Python](https://itnext.io/extracting-data-from-wikidata-using-sparql-and-python-59e0037996f)__ by Jelle van Kerkvoorde.

The query retrieves the name, location, and founding date of all cities in the United States.

In [8]:
query = '''
SELECT ?city ?cityLabel ?location ?locationLabel ?founding_date
WHERE {
?city wdt:P31/wdt:P279* wd:Q515.
?city wdt:P17 wd:Q30.
?city wdt:P625 ?location.
?city wdt:P571 ?founding_date.
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
'''

In [9]:
import python.data_extraction as DEX

data_extracter = DEX.WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
print(df.head())

                                    city  ...                       locationLabel
0     http://www.wikidata.org/entity/Q62  ...       Point(-122.416388888 37.7775)
1     http://www.wikidata.org/entity/Q62  ...       Point(-122.416388888 37.7775)
2     http://www.wikidata.org/entity/Q65  ...          Point(-118.24368 34.05223)
3   http://www.wikidata.org/entity/Q5917  ...  Point(-117.999722222 33.692777777)
4  http://www.wikidata.org/entity/Q16552  ...             Point(-117.1625 32.715)

[5 rows x 5 columns]
